In [2]:
from IPython.display import Image

지금까지 우리가 만들어온 얼굴인식 애플리케이션과 다르게 전신이 등장하는 것도 색다른 특징인데요!

나만의 카메라 앱을 스노우와 같은 얼굴인식뿐만 아니라 더 넓은 범위로 사용할 수 있다면 세상에 없던 것을 만들 수 있지 않을까요?

오늘은 새로운 기능인 human pose estimation에 대한 개념과 이론에 대해 알아보도록 하겠습니다.

Human pose estimation(HPE)은 크게 2D와 3D 로 나누어집니다.
(짧게 HPE 라고 부를게요. 공식 용어는 아닙니다.🤓)

2D HPE 는 2D 이미지에서 (x, y) 2차원 좌표들을 찾아내고, 3D HPE 는 2D 이미지에서 (x, y, z) 3차원 좌표들을 찾아내는 기술입니다.

그런데, 2차원 이미지에서 3차원 이미지를 복원하는 일은 굉장히 어려운 일입니다.



[영상 Geometry #1 좌표계](https://darkpgmr.tistory.com/77)


[영상 Geometry #7 Epipolar Geometry](https://darkpgmr.tistory.com/83?category=460965)

굉장히 어렵긴 하지만 사람의 몸은 3D 환경에서 제약이 있습니다.
(실세계 좌표계에서) 발은 바닥에 있으면서 무릎은 머리 위로 갈 수 없듯이 말이죠. 그래서 이런 제약 조건을 이용해서 어느 정도 문제를 해결할 수 있습니다.

3D pose estimation 을 깊이 있게 다루기는 매우 오랜 시간이 걸리기 때문에, 오늘은 2D 영상 내에서 (x, y) pose (관절) 의 위치를 찾는 방법을 다뤄보려 합니다.



사실 우리는 이미 2D pose estimation 을 다룬 적이 있습니다. face landmark 와 매우 비슷하지 않나요?
맞습니다. 딥러닝이나 사람의 시각에서도 실제로도 매우 비슷한 애플리케이션입니다. 입력과 출력이 개수만 다를 뿐 상당히 비슷하죠.

하지만 난이도에서 차이가 납니다.
face landmark 는 물리적으로 거의 고정되어 있는 (입이 얼굴보다 클 수 없는 걸 생각해 주세요😃) 반면, human pose 는 팔, 다리가 상대적으로 넓은 범위와 자유도를 갖는다는 것을 고려해야 합니다.

![title](https://d3s0tskafalll9.cloudfront.net/media/original_images/02_8LQHgwE.png)

자유도가 높다는 것은 데이터 분포를 특정하기 어렵다고 표현할 수 있을 것 같습니다. 데이터 분포를 학습하기 어렵다면 당연히 학습에 더 많은 데이터가 필요하고 더 복잡한 모델을 사용해야 한다는 것을 의미합니다.

따라서 상당히 많은 사전 작업이 요구되고 사용하려는 애플리케이션에 따라 접근 방법도 달라집니다.

가장 초기에 만나는 접근법은 두 가지로 나눠질 수 있습니다.

![title](https://d3s0tskafalll9.cloudfront.net/media/images/03_6C5RZR1.max-800x600.png)


#### 첫 번째 방법은 Top-down 방법입니다.

- 모든 사람의 정확한 keypoint 를 찾기 위해 object detection 을 사용합니다.


- crop 한 이미지 내에서 keypoint 를 찾아내는 방법으로 표현합니다.


- detector가 선행되어야 하고 모든 사람마다 알고리즘을 적용해야 하기 때문에 사람이 많이 등장할 때는 느리다는 단점이 있습니다.


#### 두 번째 방법은 Bottom-up 방법입니다.

- detector가 없고 keypoint 를 먼저 검출합니다.
    예를 들어 손목에 해당하는 모든 점들을 검출합니다.
    
    
- 한 사람에 해당하는 keypoint 를 clustering 합니다.


- detector 가 없기 때문에 다수의 사람이 영상에 등장하더라도 속도 저하가 크지 않습니다. 반면 top down 방식에 비해 keypoint 검출 범위가 넓어 성능이 떨어진다는 단점이 있습니다.


얼마나 정확해야 하는지, 여러 사람이 등장하는지에 따라서 필요한 알고리즘이 달라질 수 있습니다. 

핸드폰 카메라로 찍는 인물들은 대체로 소수로 등장하기 때문에 top-down 방식을 이용해도 큰 속도 저하 없이 사용할 수 있을 것이라 생각합니다.

그럼 Top-down 방법들에 대해 자세히 알아보겠습니다.

### 자유도가 높은 사람의 동작


위 챕터에서 설명한 것처럼 human pose estimation 은 keypoint 의 localization 문제를 푼다는 점에서 비슷합니다. 하지만 손목, 팔꿈치 등의 joint keypoint 정보는 얼굴의 keypoint 보다 훨씬 다양한 위치와 변화를 보입니다.

![video](https://github.com/Team-Neighborhood/Kalman-Filter-Image/raw/master/result/KF_result.gif)

위 이미지에서 볼 수 있듯이 손이 얼굴을 가리는 행위, 모든 keypoint 가 영상에 담기지 않는 등 invisible , occlusions, clothing, lighting change 가 face landmark 에 비해 더 어려운 환경을 만들어 냅니다.

딥러닝 기반 방법이 적용되기 전에는 다양한 사전 지식이 사용되었습니다.

가장 기본이 되는 아이디어는 "인체는 변형 가능 부분으로 나누어져 있고 각 부분끼리 연결성을 가지고 있다." 는 것입니다.

![title](https://d3s0tskafalll9.cloudfront.net/media/images/05_9oDmNOY.max-800x600.png)

그림에서 보이는 것처럼 손은 팔, 팔은 몸과 연결되어 있습니다. 

손이 다리 옆에 있을 확률이 팔 옆에 있을 확률보다 훨씬 작을 것입니다. 이런 제약 조건을 그림에 보이는 스프링으로 표현했습니다.

3D 환경에서 생각하면 정말 좋은 방법입니다. 

하지만 우리가 다루는 데이터는 2D 이미지 데이터이기 때문에 촬영 각도에 따라 충분히 팔이 다리 옆에서 관찰될 수 있습니다.

이 문제를 해결하기 위해 Deformable part models 방법에서는 각 부분(part) 들의 complex joint relationship 의 mixture model 로 keypoint 를 표현하는 방법을 이용했지만 성능은 사람들의 기대에 미치지 못했습니다. 자세한 내용은 아래 논문을 참고해 보세요.

[Articulated human detection with flexible mixtures-of-parts](https://www.cs.cmu.edu/~deva/papers/pose_pami.pdf)



### DeepPose

딥러닝 이전의 전통적 pose estimation 모델은 분명한 한계가 있습니다.

deformable parts model 논문에서 언급했듯이 graphical tree model 은 같은 이미지에 두 번 연산을 하는 등 연산 효율이 떨어지는 점과 그에 비해서도 부족한 성능이 문제점으로 인식되어 왔습니다.

AlexNet 이후, 다양한 분야에 CNN 이 적용되면서 pose estimation 분야에도 CNN 을 이용한 방법이 나타나기 시작했습니다.

Toshev and Szegedy 는 처음으로 딥러닝 기반 keypoint localization 모델을 제안했습니다.



![](https://d3s0tskafalll9.cloudfront.net/media/original_images/08_iBnIDWo.png)

표에서 볼 수 있듯이 DeepPose 가 전반적으로 높은 성능을 나타내고 있긴 하지만 기존 Tree based model 인 Wang et al. 의 방법에 비해 비약적으로 성능을 상승시켰다고 말하기는 어렵습니다. DeepPose 의 기여는 SOTA에 가까운 성능을 내면서도 딥러닝을 적용한 첫 번째 사례라고 할 수 있겠습니다.



### Efficient Object Localization Using Convolutional Network


DeepPose는 딥러닝을 사용했는데 왜 성능이 비약적으로 상승하지 않았을까요? 저는 이 논문이 해결책을 제시하고 있다고 생각합니다.

Tompson 이 제안한 Efficient object localization 방법을 간단하게 소개해 드리려고 합니다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/09_NSfxHyY.max-800x600.png)

이 논문에서는 제안했던 모델도 DeepPose 에 비해 깊어졌지만, 가장 중요한 건 keypoint의 위치를 직접 예측하기보다 keypoint가 존재할 확률 분포를 학습하게 하자는 점입니다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/10_SBHBawM.max-800x600.png)

human pose (keypoint) 도 사람이 labeling 을 할 수밖에 없는데 사람이 항상 같은 위치의 점을 찍을 수 있을까요? 동영상으로 보면 조금 더 쉽게 관찰할 수 있습니다.
동영상의 아래 keypoint 들을 유심히 바라봐 주세요. 귀는 귀에 눈은 눈에 그리고 어깨는 어깨에 keypoint 자체는 잘 찍혀 있는 것 같은데 어색하지 않나요?

Kalman filter 라고 적힌 동영상에 비해 Orig measured 는 점이 굉장히 떨리고 있다는 사실을 알 수 있습니다. 항상 같은 위치라고 생각하면서 keypoint 를 선택하지만 사실 매 사진마다 수 픽셀씩 차이가 생기고 있습니다. 눈을 찍고 싶다면 눈을 중심으로 "어떤 분포" 의 에러가 더해져서 저장되는 것이죠.

자연상태에서 일어나는 확률 분포는 가우시안 분포일 가능성이 큽니다. Tompson 은 이런 점에 착안하여 label 을 (x,y) 좌표에서 (x,y) 를 중심으로 하는 heatmap 으로 변환했습니다. 딥러닝 모델은 이 heatmap 을 학습하게 되는 것이죠. **"keypoint 가 존재할 확률" 을 학습하게 된 이후로 성능이 비약적으로 향상**되는 모습을 볼 수 있습니다.

#### Convolutional Pose Machines

CVPR 2016에서 발표된 CPM 은 completely differentiable 한 multi-stage 구조를 제안했습니다.

multi stage 방법들은 DeepPose 에서부터 지속적으로 사용되어 왔었습니다.


하지만 crop 연산 등 비연속적인 미분 불가능한 stage 단위로 나눠져 있었기 때문에 학습 과정을 여러 번 반복하는 비효율적인 방법을 사용해왔습니다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/13_wp3QD5J.max-800x600.png)

Stage 1 은 image feature 를 계산하는 역할을 하고 stage 2는 keypoint 를 예측하는 역할을 합니다. g1과 g2 모두 heatmap 을 출력하게 만들어서 재사용이 가능한 부분은 weight sharing 할 수 있도록 세부 모델을 설계했습니다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/14.max-800x600.png)

Stage ≥ 2 에서 볼 수 있듯이 stage 2 이상부터는 반복적으로 사용할 수 있습니다. 보통은 3개의 스테이지를 사용한다고 합니다. stage 1 구조는 고정이고 stage 2 부터는 stage 2 구조를 반복해서 추론합니다. stage 2 부터는 입력이 heatmap(image feature)이 되기 때문에 stage 단계를 거칠수록 keypoint가 refinement 되는 효과를 볼 수 있습니다.


![](https://d3s0tskafalll9.cloudfront.net/media/images/15.max-800x600.png)

사실 CPM 이 아주 좋은 방법이라고는 말하기 어렵습니다.

Multi-stage 방법을 사용하기 때문에 end-to-end 로 학습이 가능하더라도 그대로 학습하는 경우는 높은 성능을 달성하기 어렵습니다. 

따라서 stage 단위로 pretraining 을 한 후 다시 하나의 모델로 합쳐서 학습을 합니다.

논문을 작성하기 위해서라면 충분히 감내할 수 있지만 서비스 측면에서 바라본다면 불편한 요소라고 할 수 있습니다. 

이런 문제점들은 후에 제안되는 모델들이 적극적으로 개선하고 있습니다.

CPM 을 다루는 이유는 성능 때문입니다. receptive field 를 넓게 만드는 multi stage refinement 방법이 성능 향상에 크게 기여한 것 같습니다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/16.max-800x600.png)

#### Hourglass


Stacked Hourglass Network 의 기본 구조는 모래시계 같은 모양으로 만들어져 있습니다. Conv layer 와 pooling 으로 이미지(또는 feature) 를 인코딩 하고 upsampling layer 를 통해 feature map 의 크기를 키우는 방향으로 decoding 합니다. feature map 크기가 작아졌다 커지는 구조여서 hourglass 라고 표현합니다.

### SimpleBaseline

앞서 소개해드린 연구들은 딥러닝 기반의 2D human pose estimation 이 어떻게 발전해 왔는지 보여주고 있습니다. 

(x, y) 를 직접 regression 하는 방법이 heatmap 기반으로 바뀌고 모델의 구조가 바뀌어 가면서 encoder-decoder 가 쌓아져 가는 형태가 완성되었습니다.

결과적으로 MPII 에서 90% 를 넘길 정도로 좋아졌지만 모델의 구조는 다소 복잡해졌습니다.

사실 2020년 최신 논문들에 비하면 크게 복잡한 정도는 아니지만.. 18~19년 당시의 기준입니다.

HPE 의 연구를 쭉 따라오던 당시 Microsoft 인턴 Haiping Wu는 약간 다른 시각을 가져봤습니다.

"기술 자체가 많이 발전했는데 현재의 간단한 모델은 얼마나 성능이 좋을까?"

SimpleBaseline의 저자는 정말 아주 간단한 encoder-decoder 구조를 설계합니다.



오늘 본 6개의 모델이 잘 이해가 되시나요? 아마 글로만 읽어서는 크게 와닿지 않을 것 같습니다.

    A picture is worth a thousand words

저는 이 말을 참 좋아합니다.

백문이 불여일견.

시각적으로 표현했을 때 한 번에 이해할 수 있다는 점이 영상처리의 가장 큰 장점인 것 같습니다.

코드도 마찬가지입니다. 잘 작성된 코드는 하나의 예술작품처럼 아름답게 보이고 오히려 더 직관적으로 보입니다.

6개 모델 중 가장 간단한 모델인 SimpleBaseline을 모델 부분만 정확하게 이해해 보겠습니다.

### SimpleBaseline 구조

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/26.png)

- encoder : conv layers

- decoder : deconv module + upsampling

로 이루어져 있다는 것을 알 수 있지만 conv layer 가 정확히 어떻게 이루어져 있는지, deconv module 은 구성이 어떻게 되어 있는지, deconv module 이 그림처럼 3개일지는 논문을 정확하게 읽어봐야 알 수 있습니다.

논문 : [Simple Baselines for Human Pose Estimation and Tracking](https://arxiv.org/pdf/1804.06208.pdf)
